## 60. 単語ベクトルの読み込みと表示

In [15]:
from gensim.models import KeyedVectors

model: KeyedVectors = KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin.gz', binary=True)
# 実行に50秒弱かかる。

コサイン類似度について...  
要するにコサインのことらしい。
ベクトルの内積の求め方に2通りあることを利用すると簡単に算出できる。すなわち、`（ベクトルの要素同士をかけ合わせたものの合計）/（長さを全てかけ合わせたもの）＝コサイン`

もちろん実際には、ライブラリにn次元のベクトルを渡せば計算してくれる。


In [16]:
model.similarity('United_States', "U.S.")


0.73107743

In [17]:
model.similarity('United_States', "Japan")


0.52173746

In [18]:
model.similarity('United_States', "China")


0.50480795

In [19]:
model.most_similar('United_States', topn=10)


KeyboardInterrupt: 

In [ ]:
vec = model['Spain'] - model['madrid'] + model['Athens'] # スペインの首都はマドリード、ギリシャの首都はアテネ
model.most_similar(vec, topn=10)


[('Athens', 0.6826056838035583),
 ('Greece', 0.4856836199760437),
 ('Athens_Greece', 0.46644291281700134),
 ('Spain', 0.4448360204696655),
 ('Rome', 0.41419845819473267),
 ('Organising_Committee_ATHOC', 0.41101542115211487),
 ('prosecutor_Costas_Simitzoglou', 0.4097808599472046),
 ('bronze_medalist_Alicia_Molik', 0.3909006416797638),
 ('Greek', 0.39005470275878906),
 ('silver_medalist_Mardy_Fish', 0.38434845209121704)]

In [ ]:
import pandas as pd
df = pd.read_csv('./data/questions-words.txt', sep=' ', header=None, names=['vec1', 'vec2', 'vec3', 'vec4'], skiprows=1)


In [ ]:
df = df[df['vec1'].str.startswith(':') == False]

In [ ]:
from typing import Tuple

def get_most_similar_word_and_similarity(word: str) -> Tuple[str, float]:
    return model.most_similar(word, topn=1)


In [ ]:
vec1: pd.Series = df[0:10]['vec1']
df_most_similar_word_and_similarity = vec1.map(get_most_similar_word_and_similarity) # WIP: 本当は2列にしたかった


In [ ]:
semantic = df[df.index <= 8872] # 手抜き
syntactic = df[df.index > 8872]


In [ ]:
word_sim_df = pd.read_csv('./data/wordsim353/set1.csv', sep=',')


In [43]:
def get_similarity(row):
    # print(row['Word 1'], row['Word 2'])
    return model.similarity(row['Word 1'], row['Word 2'])


In [45]:
word_sim_df['Machine'] = word_sim_df[['Word 1', 'Word 2',]].apply(lambda row: get_similarity(row), axis=1)
word_sim_df.head()

,Word 1,Word 2,Human (mean),1,2,3,4,5,6,7,8,9,10,11,12,13,Machine
0,love,sex,6.77,9.0,6.0,8.0,8,7,8,8.0,4,7.0,2,6.0,7,8,0.263938
1,tiger,cat,7.35,9.0,7.0,8.0,7,8,9,8.5,5,6.0,9,7.0,5,7,0.517296
2,tiger,tiger,10.00,10.0,10.0,10.0,10,10,10,10.0,10,10.0,10,10.0,10,10,1.000000
3,book,paper,7.46,8.0,8.0,7.0,7,8,9,7.0,6,7.0,8,9.0,4,9,0.363463
4,computer,keyboard,7.62,8.0,7.0,9.0,9,8,8,7.0,7,6.0,8,10.0,3,9,0.396392


In [48]:
word_sim_df[['Human (mean)', 'Machine']].corr(method='spearman')

,Human (mean),Machine
Human (mean),1.000000,0.665139
Machine,0.665139,1.000000
